This code will most likely not work properly in VSCode. It is a slightly adapted version of what was used in Google Colab.

In [1]:
# Import Necessary Libraries
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import multivariate_normal, bernoulli, beta, norm
from scipy.stats import ks_2samp
from scipy.special import expit as logistic_sigmoid
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score, brier_score_loss, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import pickle

np.random.seed(42)

In [2]:
df = pd.read_csv('../regression/test_dataset.csv', index_col=False)
model = pickle.load(open('../regression/regression_model_MM.sav', 'rb'))
model_weighed = pickle.load(open('../regression/regression_model_Bias.sav', 'rb'))
model_weighed_y = pickle.load(open('../regression/regression_model_Bias2.sav', 'rb'))

In [3]:
# Prepare the features and labels
features = df.drop(columns=['fake_binary'])
labels = df['fake_binary']

# Create a combined DataFrame to pass to WIT
wit_data = features.copy()
wit_data['fake_binary'] = labels

binary_variable = 'party_detailed'  # Replace with your binary variable name

# Define a function to make predictions using the loaded models
class CustomModelWrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, inputs):
        inputs_df = pd.DataFrame(inputs, columns=features.columns)
        predicted_probs = self.model.predict(inputs_df)
        return np.expand_dims(predicted_probs, axis=1)

# Instantiate the wrappers with your models
custom_model = CustomModelWrapper(model)
custom_model_weighed = CustomModelWrapper(model_weighed)
custom_model_weighed_y = CustomModelWrapper(model_weighed_y)


# Prepare WIT configuration for the first model
config_builder1 = WitConfigBuilder(wit_data.values.tolist(), wit_data.columns.tolist())
config_builder1.set_custom_predict_fn(custom_model.predict)
config_builder1.set_target_feature('fake_binary')

# Prepare WIT configuration for the second model
config_builder2 = WitConfigBuilder(wit_data.values.tolist(), wit_data.columns.tolist())
config_builder2.set_custom_predict_fn(custom_model_weighed.predict)
config_builder2.set_target_feature('fake_binary')

# Prepare WIT configuration for the third model
config_builder3 = WitConfigBuilder(wit_data.values.tolist(), wit_data.columns.tolist())
config_builder3.set_custom_predict_fn(custom_model_weighed_y.predict)
config_builder3.set_target_feature('fake_binary')


In [4]:
wit1 = WitWidget(config_builder1)

display(wit1)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'feature_names': ['const', 'statuses_coun…

In [5]:
wit2 = WitWidget(config_builder2)

display(wit2)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'feature_names': ['const', 'statuses_coun…

In [6]:
wit3 = WitWidget(config_builder3)

display(wit3)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'feature_names': ['const', 'statuses_coun…

In [7]:
# Predict using each model independently
predictions1 = custom_model.predict(features.values)
predictions2 = custom_model_weighed.predict(features.values)
predictions3 = custom_model_weighed_y.predict(features.values)

# Convert probabilities to binary predictions
binary_predictions1 = (predictions1 >= 0.5).astype(int)
binary_predictions2 = (predictions2 >= 0.5).astype(int)
binary_predictions3 = (predictions3 >= 0.5).astype(int)

# Print predictions for verification
print("Model 1 Predictions:", binary_predictions1[:10])
print("Model 2 Predictions:", binary_predictions2[:10])
print("Model 3 Predictions:", binary_predictions3[:10])

# Calculate accuracy for each model
accuracy1 = accuracy_score(labels, binary_predictions1)
accuracy2 = accuracy_score(labels, binary_predictions2)
accuracy3 = accuracy_score(labels, binary_predictions3)

print(f"Accuracy Model 1: {accuracy1}")
print(f"Accuracy Model 2: {accuracy2}")
print(f"Accuracy Model 3: {accuracy3}")


Model 1 Predictions: [[0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]
Model 2 Predictions: [[0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]]
Model 3 Predictions: [[0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]]
Accuracy Model 1: 0.7395833333333334
Accuracy Model 2: 0.7458333333333333
Accuracy Model 3: 0.6645833333333333
